In [49]:
import requests, json

def get_bybit_price(symbol):
    """Get current spot price from Bybit"""
    url = "https://api.bybit.com/v5/market/tickers"
    params = {"category": "spot", "symbol": symbol}
    response = requests.get(url, params=params)
    data = response.json()
    return float(data["result"]["list"][0]["lastPrice"])


In [50]:
def get_jupiter_price(input_mint, output_mint, amount):
    """Get Jupiter price for one mint pair"""
    url = "https://lite-api.jup.ag/swap/v1/quote"
    params = {
        "inputMint": input_mint,
        "outputMint": output_mint,
        "amount": amount
    }

    response = requests.get(url, params=params)
    data = response.json()
    
    if "error" in data:
        return None
    else:
        return int(data["outAmount"]) / int(data["inAmount"])


In [51]:
def get_best_jupiter_price(bybit_symbol, amount):

    with open("../files/monitor_pool.json", "r") as f:
        trading_pairs = json.load(f)

    for pair in trading_pairs:
        if pair["bybit_symbol"] == bybit_symbol:
            base_mints = pair["base_token"]["jupiter_mints"]
            quote_mint = pair["quote_token"]["jupiter_mints"][0]["mint"]
            
            # Get all prices
            results = []
            for mint in base_mints:
                price = get_jupiter_price(mint["mint"], quote_mint, amount)
                if price is not None:
                    results.append({"mint": mint["mint"], "price": price})
            return results
    
    return []

print(get_bybit_price("TRUMPUSDT"))
print(get_best_jupiter_price("TRUMPUSDT", 10))

6.83
[{'mint': '6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN', 'price': 6.8}]


In [52]:
def get_matches(bybit_symbol):

    with open("../files/monitor_pool.json", "r") as f:
        trading_pairs = json.load(f)

    for pair in trading_pairs:
        if pair["bybit_symbol"] == bybit_symbol:
            base_mints = pair["base_token"]["jupiter_mints"]
            quote_mint = pair["quote_token"]["jupiter_mints"][0]["mint"]
            
            # Get all prices
            results = []
            for mint in base_mints:
                results.append(mint["mint"])
            
            return results
    
    return []

In [53]:
results = get_matches('TRUMPUSDT')
print(results)
for result in results:
    print(result)
    print(get_jupiter_price(result,'Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB'))
    print('-'*10)

['6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN', 'HaP8r3ksG76PhQLTqR8FYBeNiQpejcFbQmiHbg787Ut1', 'AwRErBEFGTnohzfLeRSBH9HddQEy2oeRxnWLrbvFFh95', '4h8LjZWUfUQVgbEZ29UzTuGXNW6rwrJis78ZU66ekkPV', 'BqhNdGtS1Nqtzi2MvZ7G8NN1vRuHZ12UpHGJKe71e1JT']
6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN


TypeError: get_jupiter_price() missing 1 required positional argument: 'amount'